# Smart Home Controller with Generative AI

## Introduction

This project simulates a smart home environment where users can control and monitor virtual devices using natural language commands. Instead of relying on buttons or menus, the system allows users to simply type commands like “Turn on the light” or “Set the fan speed to high.” These commands are processed using a Generative AI model (OpenAI GPT), which interprets the intent and translates it into actions.

The system includes simulations of three common smart home devices:

- Light: Can be turned ON or OFF.
- Fan: Can be turned ON or OFF and have its speed set to *low*, *medium*, or *high*.
- Thermostat: Allows setting a specific temperature, between 18°C and 30°C.

This project demonstrates the potential of AI to create intuitive, conversational interfaces for everyday tasks, while also providing a hands-on example of combining Python logic with language models for real-world applications.


## Step 1: Simulating Smart Home Devices

In this step, we simulate three common smart home devices — a Light, a Fan, and a Thermostat — using Python classes. Each class represents a device with specific functionalities and maintains its own internal state.

### Light
- Can be turned `ON` or `OFF`.
- Includes methods to change its state and retrieve its current status.

### Fan
- Can be turned `ON` or `OFF`.
- Supports speed settings: `low`, `medium`, or `high`.
- The fan's speed can be adjusted only to valid values.

### Thermostat
- Allows setting a temperature between 18°C and 30°C.
- Stores the current temperature setting and returns it on request.


In [14]:
# Step 1: Simulate Smart Home Devices

class Light:
    def __init__(self):
        # Initial state is OFF
        self.state = "OFF"

    def turn_on(self):
        self.state = "ON"

    def turn_off(self):
        self.state = "OFF"

    def get_status(self):
        return f"The light is {self.state}."


class Fan:
    def __init__(self):
        # Initial state is OFF and speed is set to low by default
        self.state = "OFF"
        self.speed = "low"

    def turn_on(self):
        self.state = "ON"

    def turn_off(self):
        self.state = "OFF"

    def set_speed(self, speed):
        # Allow only valid speeds if fan is ON
        if speed in ["low", "medium", "high"]:
            self.speed = speed

    def get_status(self):
        return f"The fan is {self.state} with speed set to {self.speed}."


class Thermostat:
    def __init__(self):
        # Default temperature is 22°C
        self.temperature = 22

    def set_temperature(self, temp):
        # Only allow temperature within 18–30°C range
        if 18 <= temp <= 30:
            self.temperature = temp

    def get_status(self):
        return f"The thermostat is set to {self.temperature}°C."

## Step 2: Command Parsing with Generative AI

To make the system understand natural language commands, we use a Generative AI model — specifically, `gpt-3.5-turbo` from OpenAI.

Instead of manually coding every possible command, we use AI to interpret user input and convert it into structured, machine-readable JSON. This allows for flexible and natural interaction with the system.

### Example

- User Input:  
  `Turn on the light`

- AI Output:
  ```json
  {
    "device": "light",
    "action": "turn_on"
  }


In [ ]:
import os

# Set your API key securely (this will not be stored)
os.environ['OPENAI_API_KEY'] = input("Paste your OpenAI API key: ")


In [20]:
import openai

# Use the key from environment
openai.api_key = os.getenv("OPENAI_API_KEY")


In [21]:
# Step 2: Command Parsing with OpenAI GPT

def parse_command(user_input):
    """
    Uses GPT to parse a natural language command and return a structured response.
    The expected format is a JSON object with fields:
    - device: "light", "fan", or "thermostat"
    - action: e.g., "turn_on", "turn_off", "set_speed", "set_temperature", "get_status"
    - value: optional (e.g., speed or temperature)
    """
    prompt = f"""
You are a smart home assistant. Interpret the user's command and convert it to JSON format like:
{{
    "device": "light/fan/thermostat",
    "action": "turn_on/turn_off/set_speed/set_temperature/get_status",
    "value": "optional"
}}

Command: "{user_input}"
Response:
"""
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )

    # Return only the content of the model's reply (the JSON string)
    return response['choices'][0]['message']['content']

## Step 3: Command Execution and Feedback

In this step, we take the structured JSON output from the AI model and use it to control the corresponding smart home devices. The function `execute_command()` is responsible for this logic.

###How It Works

1. The AI returns a command in JSON format, like:
   ```json
   {
     "device": "fan",
     "action": "set_speed",
     "value": "high"
   }


In [22]:
import json

def execute_command(devices, parsed_json):
    """
    Executes a parsed command on the smart devices.

    Parameters:
    - devices: dict of device objects (light, fan, thermostat)
    - parsed_json: JSON string or dictionary from parse_command()

    Returns:
    - A message to display to the user
    """
    try:
        # Convert string to dictionary if needed
        if isinstance(parsed_json, str):
            data = json.loads(parsed_json)
        else:
            data = parsed_json

        device = data.get("device")
        action = data.get("action")
        value = data.get("value", None)

        if device == "light":
            if action == "turn_on":
                devices["light"].turn_on()
                return "The light is now ON."
            elif action == "turn_off":
                devices["light"].turn_off()
                return "The light is now OFF."
            elif action == "get_status":
                return devices["light"].get_status()

        elif device == "fan":
            if action == "turn_on":
                devices["fan"].turn_on()
                return "The fan is now ON."
            elif action == "turn_off":
                devices["fan"].turn_off()
                return "The fan is now OFF."
            elif action == "set_speed":
                devices["fan"].set_speed(value)
                return f"The fan speed is set to {value}."
            elif action == "get_status":
                return devices["fan"].get_status()

        elif device == "thermostat":
            if action == "set_temperature":
                devices["thermostat"].set_temperature(int(value))
                return f"The thermostat is set to {value}°C."
            elif action == "get_status":
                return devices["thermostat"].get_status()

        elif device == "all" and action == "get_status":
            return (
                devices["light"].get_status() + " " +
                devices["fan"].get_status() + " " +
                devices["thermostat"].get_status()
            )


        return "Command not recognized."

    except Exception as e:
        return f"Error: {str(e)}"

> Includes error handling to deal with unexpected input or invalid commands.

### Benefits

- Modular: Any future devices or actions can be added easily.
- Flexible: Supports both string and dictionary JSON input.
- Safe: Gracefully handles errors and unsupported actions.

This layer connects the language-understanding power of the AI with the device logic coded in Python, enabling a seamless natural language interface for controlling smart devices.

## Step 4: Command-Line Interface (CLI)

In this final step, we create a simple and intuitive command-line interface (CLI) that allows users to interact with the smart home system in real time using natural language commands.

### How It Works

1. The system initializes the three smart devices: `Light`, `Fan`, and `Thermostat`.
2. The user types a natural language command (e.g., "Turn on the fan").
3. The command is passed to the `parse_command()` function, which uses GPT to return a structured JSON.
4. The `execute_command()` function interprets the JSON and performs the correct action.
5. The system responds with a user-friendly message showing the result of the command.

This loop continues until the user types `exit`.

In [23]:
def main():
    # Initialize the simulated devices
    devices = {
        "light": Light(),
        "fan": Fan(),
        "thermostat": Thermostat()
    }

    print("Welcome to your Smart Home!")
    print("Type a command (or type 'exit' to quit)\n")

    while True:
        # Get input from the user
        user_input = input("You: ")

        if user_input.lower() == "exit":
            print("Exiting Smart Home.")
            break

        # Parse the command using GPT
        parsed = parse_command(user_input)
        print("AI Parsed:", parsed)

        # Execute the command on the device
        feedback = execute_command(devices, parsed)
        print("Smart Home:", feedback)

- The command is passed to the AI parser (`parse_command()`), which
converts it to structured JSON.
- The JSON is processed by the execution logic (`execute_command()`), which performs the corresponding action on the device.
- A human-readable response is printed, such as:

### Example Interaction

- **Smart Home**: Welcome to your Smart Home! Type a command (or type 'exit' to quit)

- **You**: Set the fan speed to high

- **AI Parsed**: {"device": "fan", "action": "set_speed", "value": "high"}

- **Smart Home**: The fan speed is set to high.

- **You**: What is the temperature?

- **AI Parsed**: {"device": "thermostat", "action": "get_status"}

- **Smart Home**: The thermostat is set to 22°C.